In [1]:
import re
import urllib
import cx_Oracle
import urllib.request
import os
import pandas as pd
import numpy as np

In [2]:
# pre_process
#import pandas as pd
#import numpy as np
#label=['date','ticker','c_price','pe','pb','ps']
#csv_name=['value_factor06-07.csv','value_factor08-09.csv','value_factor10-11.csv','value_factor12-13.csv','value_factor14-15.csv','value_factor16-17.csv','value_factor18-19.csv']
#df_frame=[]
#for a in csv_name:
   #df=pd.read_csv(a,usecols=[1,2,3,4,5,6],names=label,header=None)
    #df_frame.append(df)
#df_valfac=pd.concat(df_frame)

# Get data

In [3]:
label=['date','ticker','val_mv','mv','c_price','pe','pb','ps','turn','oper_rev']
df_valfac=pd.read_csv('EOD_factorAll.csv',usecols=[1,2,3,4,5,6,7,8,9,10],names=label,header=None)

In [4]:
df_valfac.head()

,date,ticker,val_mv,mv,c_price,pe,pb,ps,turn,oper_rev
0,20060101,600131.SH,173923.1785,58620.6802,3.45,46.2222,2.3344,2.9900,NaN,5.816855e+08
1,20060101,600114.SH,106547.5000,41692.5000,5.45,11.0670,1.7441,2.4251,NaN,4.393576e+08
2,20060101,600762.SH,15426.9440,6316.5440,1.46,NaN,6.6396,NaN,NaN,-9.437332e+07
3,20060101,600755.SH,149778.7192,96011.9992,4.20,9.7317,1.5992,0.1281,NaN,1.169415e+10
4,20060101,600090.SH,112582.4937,53281.0138,3.06,NaN,NaN,6.5737,NaN,1.712629e+08


In [5]:
# get daily closed prices of three stocks from raw data
df_price=df_valfac[['date','ticker','c_price']]
target=['000001.SZ','600889.SH',"600090.SH"]
df_c=df_price.loc[df_price['ticker'].isin(target)]

# Cleanning data and calculate monthly return

In [6]:
# change the type of date
df_c['date'] = df_c['date'].apply(str)

# length to width
df_c_trans=df_c.pivot(
"date",    #行索引（可以使多个类别变量）
"ticker",                   #列索引（可以使多个类别变量）
"c_price"                      #值（一般是度量指标）
     )

#dft1.index=(dft1.index).apply(str)
df_c_trans.index=pd.to_datetime(df_c_trans.index)
dft1=df_c_trans.apply(lambda x:x.resample('M').last())
dft1

dft2=df_c_trans.apply(lambda x: x.resample('M').first())
dft2

df_Mon_PChg=dft1/dft2-1
df_Mon_PChg.head()

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


ticker,000001.SZ,600090.SH,600889.SH
date,,,
2006-01-31,0.034202,0.000000,0.110345
2006-02-28,0.077165,0.039216,0.124224
2006-03-31,-0.060472,0.057508,-0.041667
2006-04-30,0.237049,0.063444,-0.066667
2006-05-31,0.114213,0.261364,0.422360


# Set constraint 

In [7]:
def getdata():
    a=np.array(df_Mon_PChg)
    return a 
    

In [8]:
def solver_constraint_1(t):
    return t[2]+t[1]+t[0]-1

x_cons = ({'type': 'eq', 'fun': solver_constraint_1})

In [9]:
#def weigdrawback(df_Mon_PChg,w1,w2,w3):
    #df_Mon_PChg['weighted']=w1*df_Mon_PChg.iloc[:,0]+w2*df_Mon_PChg.iloc[:,1]+w3*df_Mon_PChg.iloc[:,2]
    #return -df_Mon_PChg['weighted'].min()

# Minimize Conditional value at risk

# Maximize return

In [10]:
def minreturn(x0):
    a=getdata()
    w1,w2,w3=x0
    wd=[]
    for i in range(0,len(a)):
        wd.append((w1*a[i][0]+w2*a[i][1]+w3*a[i][2]))
    x=1
    for j in range(0,len(wd)):
        x=x*(wd[j]+1)
    return -x
    

In [11]:
#test for minreturn function
x0=[0.06,0.86,0.07]
minreturn(x0)

-2.231615689882035

In [12]:
from scipy.optimize import minimize
x0=[0.1,0.2,0.7]
a=np.array(df_Mon_PChg)
#cons=({'type': 'eq', 'fun': w1+w2+w3-1})
y_minre=minimize(minreturn,x0,bounds=((0,1),(0,1),(0,1)),constraints=x_cons)

In [13]:
y_minre

     fun: -3.636239007847186
     jac: array([-0.26310074, -0.26330513, -0.26315799])
 message: 'Optimization terminated successfully.'
    nfev: 27
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([0.3736528 , 0.24074336, 0.38560385])

# Minimize Variance

In [14]:
# whether variance is the squared volatility
def minvar(x0):
    a=getdata()
    w1,w2,w3=x0
    wd=[]
    for i in range(0,len(a)):
        wd.append((w1*a[i][0]+w2*a[i][1]+w3*a[i][2]))
    return np.var(wd)
    

In [15]:
#test for minvar function
x0=[0.06,0.87,0.07]
minvar(x0)

0.025533350408209136

In [16]:
from scipy.optimize import minimize
x0=[0.1,0.2,0.7]
a=np.array(df_Mon_PChg)
#cons=({'type': 'eq', 'fun': w1+w2+w3-1})
y_var=minimize(minvar,x0,bounds=((0,1),(0,1),(0,1)),constraints=x_cons)

In [17]:
y_var

     fun: 0.01308731258609668
     jac: array([0.02617481, 0.02617435, 0.02617438])
 message: 'Optimization terminated successfully.'
    nfev: 45
     nit: 9
    njev: 9
  status: 0
 success: True
       x: array([0.57553575, 0.16261972, 0.26184453])

# Minimize volatility

In [18]:
def volatility(x0):
    a=getdata()
    w1,w2,w3=x0
    wd=[]
    for i in range(0,len(a)):
        wd.append((w1*a[i][0]+w2*a[i][1]+w3*a[i][2]))
    return -np.nanstd(wd)
    

In [19]:
#test for volatility function
x0=[0.06,0.07,0.87]
volatility(x0)

-0.13734921484844534

In [20]:
from scipy.optimize import minimize
x0=[0.1,0.2,0.7]
a=np.array(df_Mon_PChg)
#cons=({'type': 'eq', 'fun': w1+w2+w3-1})
y_vol=minimize(volatility,x0,bounds=((0,1),(0,1),(0,1)),constraints=x_cons)

In [21]:
y_vol

     fun: -0.14497624578099766
     jac: array([-0.06325249, -0.09781086, -0.14497625])
 message: 'Optimization terminated successfully.'
    nfev: 30
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([0.00000000e+00, 2.22044605e-16, 1.00000000e+00])

# Minimize maximum drawback

In [22]:
def weigdrawback(x0):
    a=getdata()
    w1,w2,w3=x0
    wd=[]
    for i in range(0,len(a)):
        wd.append((w1*a[i][0]+w2*a[i][1]+w3*a[i][2]))
    return -min(wd)

In [23]:
#test for weigdrawback function
x0=[0.07,0.86,0.07]
weigdrawback(x0)

0.412070835399529

In [24]:
from scipy.optimize import minimize
x0=[0.1,0.2,0.7]
a=np.array(df_Mon_PChg)
#cons=({'type': 'eq', 'fun': w1+w2+w3-1})
y_mindb=minimize(weigdrawback,x0,bounds=((0,1),(0,1),(0,1)),constraints=x_cons)

In [25]:
y_mindb

     fun: 0.34774894930947486
     jac: array([0.44162775, 0.3       , 0.43128964])
 message: 'Optimization terminated successfully.'
    nfev: 69
     nit: 12
    njev: 12
  status: 0
 success: True
       x: array([3.37140232e-01, 6.62859768e-01, 1.61278215e-15])

# Maximize sharp rate

In [26]:
# for the convenience of test we set the risk free rate as a fixed number
def sharprate(x0):
    a=getdata()
    w1,w2,w3=x0
    wd=[]
    for i in range(0,len(a)):
        wd.append((w1*a[i][0]+w2*a[i][1]+w3*a[i][2]))
    sr=(np.mean(wd)-0.02/12)/np.std(wd)
    return -sr

In [27]:
#test for functuon sharprate
x0=[0.07,0.86,0.07]
v=sharprate(x0)
v

-0.09033374409648143

In [28]:
from scipy.optimize import minimize
x0=[0.1,0.2,0.7]
a=np.array(df_Mon_PChg)
#cons=({'type': 'eq', 'fun': w1+w2+w3-1})
y_sr=minimize(sharprate,x0,bounds=((0,1),(0,1),(0,1)),constraints=x_cons)

In [29]:
y_sr

     fun: -0.11381653895386679
     jac: array([-0.01419948, -0.01423913, -0.014335  ])
 message: 'Optimization terminated successfully.'
    nfev: 50
     nit: 10
    njev: 10
  status: 0
 success: True
       x: array([0.3962072 , 0.20709289, 0.39669991])